In [1]:
# Import the Earth Engine Python Package
import ee

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

In [70]:
# Import the required Images and ImageCollections
sn = ee.FeatureCollection("ft:1vdDUTu09Rkw5qKR_DSfmFX-b_7kqy4E-pjxg9Sq6");
modis = ee.ImageCollection("MODIS/006/MOD13Q1");

# How many images are in the MODIS ImageCollection?
print(modis.size().getInfo());

397


In [71]:
# Display an image to make sure everything is working as expected.

from IPython.display import Image, display, HTML

thumburl = ee.Image(modis.first()).select('EVI').getThumbUrl({
                'min':0, 
                'max':8000          
            })

#print thumburl
img_thumb = Image(url=thumburl)
display(img_thumb)

In [72]:
# Define the masking function that will determine which pixels are both forested AND of high MODIS quality.

def quality_mask(img):
  date = ee.Image(ee.Number(img.get('system:time_start'))).rename(["date"]).divide(1000)
  mask = ee.Image(1).rename(["mask"])
     
  quality_pixels = img.select(["SummaryQA"]).rename(["quality_pixel"]).eq(0)
  
  export_img = img.select(["EVI"]).addBands(date).addBands(mask).updateMask(quality_pixels).toInt32().clip(sn)

  return export_img;

In [73]:
# Map the masking function to the MODIS image collection

m_modis = modis.map(quality_mask);
print(ee.Image(m_modis.first()).getInfo())
print(ee.Image(m_modis.first()).bandNames().getInfo())

{'properties': {'system:index': '2000_02_18', 'system:asset_size': 28219636986, 'system:time_start': 950832000000, 'system:time_end': 952214400000}, 'version': 1493667561435000, 'type': 'Image', 'id': 'MODIS/006/MOD13Q1/2000_02_18', 'bands': [{'crs_transform': [231.65635826395825, 0.0, -20015109.353988, 0.0, -231.65635826395834, 10007554.676994], 'data_type': {'precision': 'int', 'max': 2147483647, 'type': 'PixelType', 'min': -2147483648}, 'crs': 'SR-ORG:6974', 'id': 'EVI', 'dimensions': [172800, 86400]}, {'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'data_type': {'precision': 'int', 'max': 2147483647, 'type': 'PixelType', 'min': -2147483648}, 'crs': 'EPSG:4326', 'id': 'date'}, {'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'data_type': {'precision': 'int', 'max': 2147483647, 'type': 'PixelType', 'min': -2147483648}, 'crs': 'EPSG:4326', 'id': 'mask'}]}
['EVI', 'date', 'mask']


In [74]:
# Test the first image
from IPython.display import Image, display, HTML

test_img = ee.Image(m_modis.toList(1, 0).get(0))
date = ee.Date(img.get('system:time_start')).format("YYYYMMdd")

thumburl = test_img.getThumbUrl({
                'min':0, 
                'max':8000,
            })

#print thumburl
img_thumb = Image(url=thumburl)
display(img_thumb)

In [ ]:
# loop through all images and create a feature representing the image number and acquisition date
# Export each image upon looping

# There are 397 images as of 2017-06-04
# print(m_modis.size().getInfo());

first_img = 0
num_imgs = 397

# Loop through all images, add the date to the appropriate list element, and export image
for i in range(first_img, num_imgs):
    img = ee.Image(m_modis.toList(1, i).get(0));
    date = ee.Date(img.get('system:time_start')).format("YYYYMMdd");
  
    task = ee.batch.Export.image.toDrive(**{
        'image': img, 
        'description': "sn_jep_modis_ts_quality_mask_epsg3310_" + str(date.getInfo()), 
        'folder': 'ee/sn_jep_modis_ts_quality_mask_epsg3310', 
        'scale': 250, 
        'region': sn.geometry().getInfo()['coordinates'], 
        'crs': 'EPSG:3310'
    })
    task.start()